# IoT Weather Station Code 

Involves Data Preprocessing & Initial ML development

In [1]:
%pip install pandas numpy matplotlib seaborn scikit-learn plotly pyarrow fastparquet xgboost torch


[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.metrics import classification_report, mean_squared_error, r2_score
from sklearn.ensemble import IsolationForest, GradientBoostingRegressor, StackingRegressor
from sklearn.multioutput import MultiOutputClassifier
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import Ridge, Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
import plotly.express as px
import plotly.figure_factory as ff 
import math
import pathlib
from pathlib import Path
import os

In [3]:
#getting the directory of notebook 

cwd = os.getcwd()

print(f"Current Working Directory: {cwd}")

Current Working Directory: /Users/saikeerthan/Coding/NYP/IOTA/IoT_Weather_project/model-training/code


In [4]:
DATA_ROOT = Path("/Users/saikeerthan/Coding/NYP/IOTA/IoT_Weather_project/model-training/datasets/official_data.csv")

print(DATA_ROOT.exists())

True


## Data Processing 

In [5]:
# retrieving the dataset

df = pd.read_csv(DATA_ROOT) 

df


,time,cidx,cattr,temp,humi,pres,windspeed,winddirection,rainfall,uvindex
0,2026-01-24 20:23:02,1,7,28.6,74,1016.746582,5.5,54,NaN,NaN
1,2026-01-24 20:28:02,2,7,28.5,74,1016.802490,5.3,53,NaN,NaN
2,2026-01-24 20:33:02,3,7,28.6,74,1016.859131,5.0,53,NaN,NaN
3,2026-01-24 20:38:02,4,7,28.5,74,1016.934326,4.9,50,NaN,NaN
4,2026-01-24 20:43:02,5,7,28.6,74,1016.971436,5.4,50,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2271,2026-02-02 10:26:15,1045,7,38.5,47,1018.157959,6.9,75,NaN,2.0
2272,2026-02-02 10:31:15,1046,7,38.5,47,1018.206055,8.2,73,NaN,2.0
2273,2026-02-02 10:36:15,1047,7,38.5,47,1018.149414,8.2,66,NaN,2.0
2274,2026-02-02 10:41:15,1048,7,38.5,47,1018.127930,8.1,47,NaN,2.0


In [6]:
#checking for duplicates 

duplicates = df.duplicated().sum()

print(f"Duplicates present in DF: {duplicates}")

Duplicates present in DF: 0


In [7]:
# checking for missing values 

missing = df.isnull().sum().sum()

print(f"Missing values in df: {missing}")

# missing values is because of last two columns

Missing values in df: 3768


In [8]:
#columns in df 

columns = df.columns

print("Columns in Dataset:\n")
for i, col in enumerate(df.columns, 1):
    print(f"{i}. {col}")

Columns in Dataset:

1. time
2. cidx
3. cattr
4. temp
5. humi
6. pres
7. windspeed
8. winddirection
9. rainfall
10. uvindex


In [9]:
# drop entries where cattr is below 7

df = df[df["cattr"] >=7]

df

,time,cidx,cattr,temp,humi,pres,windspeed,winddirection,rainfall,uvindex
0,2026-01-24 20:23:02,1,7,28.6,74,1016.746582,5.5,54,NaN,NaN
1,2026-01-24 20:28:02,2,7,28.5,74,1016.802490,5.3,53,NaN,NaN
2,2026-01-24 20:33:02,3,7,28.6,74,1016.859131,5.0,53,NaN,NaN
3,2026-01-24 20:38:02,4,7,28.5,74,1016.934326,4.9,50,NaN,NaN
4,2026-01-24 20:43:02,5,7,28.6,74,1016.971436,5.4,50,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2271,2026-02-02 10:26:15,1045,7,38.5,47,1018.157959,6.9,75,NaN,2.0
2272,2026-02-02 10:31:15,1046,7,38.5,47,1018.206055,8.2,73,NaN,2.0
2273,2026-02-02 10:36:15,1047,7,38.5,47,1018.149414,8.2,66,NaN,2.0
2274,2026-02-02 10:41:15,1048,7,38.5,47,1018.127930,8.1,47,NaN,2.0


In [10]:
columns_to_delete = ["cattr", "windspeed", "winddirection", "rainfall", "uvindex", "cidx"]

df = df.drop(columns=columns_to_delete)

columns = df.columns
print(f"Remaining Columns:{columns}")

Remaining Columns:Index(['time', 'temp', 'humi', 'pres'], dtype='object')


In [11]:
# check the dtype of every column in the df 

print(df.dtypes)

time     object
temp    float64
humi      int64
pres    float64
dtype: object


In [12]:
df["time"] = pd.to_datetime(df["time"])

print(f"New Dtypes for columns in df: {df.dtypes}")

New Dtypes for columns in df: time    datetime64[ns]
temp           float64
humi             int64
pres           float64
dtype: object


In [13]:

# Convert to datetime safely
df["time"] = pd.to_datetime(df["time"], errors="coerce")

# Drop invalid timestamps
df = df.dropna(subset=["time"])

# Sort chronologically (CRITICAL for time-series ML)
df = df.sort_values("time").reset_index(drop=True)

# (Recommended) Set as index if doing time-based operations later
df = df.set_index("time")

# Extract time features (daily patterns help weather prediction)
df["hour"] = df.index.hour
df["dayofweek"] = df.index.dayofweek

# Cyclic encoding (prevents 23 → 0 discontinuity)
df["hour_sin"] = np.sin(2 * np.pi * df["hour"] / 24)
df["hour_cos"] = np.cos(2 * np.pi * df["hour"] / 24)


In [14]:
df

,temp,humi,pres,hour,dayofweek,hour_sin,hour_cos
time,,,,,,,
2026-01-24 20:23:02,28.6,74,1016.746582,20,5,-0.866025,0.500000
2026-01-24 20:28:02,28.5,74,1016.802490,20,5,-0.866025,0.500000
2026-01-24 20:33:02,28.6,74,1016.859131,20,5,-0.866025,0.500000
2026-01-24 20:38:02,28.5,74,1016.934326,20,5,-0.866025,0.500000
2026-01-24 20:43:02,28.6,74,1016.971436,20,5,-0.866025,0.500000
...,...,...,...,...,...,...,...
2026-02-02 10:26:15,38.5,47,1018.157959,10,0,0.500000,-0.866025
2026-02-02 10:31:15,38.5,47,1018.206055,10,0,0.500000,-0.866025
2026-02-02 10:36:15,38.5,47,1018.149414,10,0,0.500000,-0.866025


In [15]:
# save the new df 

df.to_csv("df_preprocessed.csv")

In [16]:
%pip install -U pyarrow
%pip instlal -U fastparquet


[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
ERROR: unknown command "instlal" - maybe you meant "install"
Note: you may need to restart the kernel to use updated packages.


In [17]:
WEATHER_COLS = ["temp", "humi", "pres"]

for col in WEATHER_COLS:
    df[f"{col}_lag1"] = df[col].shift(1)
    df[f"{col}_lag2"] = df[col].shift(2)
    df[f"{col}_lag3"] = df[col].shift(3)

# -----------------------------
# 5️⃣ CREATE NEXT-HOUR TARGETS
# -----------------------------
# ⚠ Adjust this based on sampling rate
# If data every 5 min → 12 steps = 1 hour
# If data every 10 min → 6 steps
# If data hourly → 1 step

SHIFT_STEPS = 12   # ← CHANGE IF NEEDED

for col in WEATHER_COLS:
    df[f"{col}_next1h"] = df[col].shift(-SHIFT_STEPS)

# -----------------------------
# 6️⃣ REMOVE NAN ROWS FROM SHIFTS
# -----------------------------
df = df.dropna()

# -----------------------------
# 7️⃣ OPTIONAL: DROP RAW HOUR IF USING SIN/COS
# -----------------------------
# df = df.drop(columns=["hour"])

# -----------------------------
# 8️⃣ SAVE ML-READY DATASET
# -----------------------------
df.to_parquet("weather_ml_ready.parquet")
df.to_csv("weather_ml_ready.csv")

print("✅ ML-ready dataset created")
print(df.shape)

✅ ML-ready dataset created
(2182, 19)


## Model Training Code 

Model Name - Climatrix

In [4]:
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge, ElasticNet
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

import pathlib
from pathlib import Path

In [2]:
print(f"Current Working Directory: {os.getcwd()}")

Current Working Directory: /Users/saikeerthan/Coding/NYP/IOTA/IoT_Weather_project/model-training/code


In [5]:
ML_DATA_ROOT = Path("/Users/saikeerthan/Coding/NYP/IOTA/IoT_Weather_project/model-training/datasets/weather_ml_ready.csv")

if ML_DATA_ROOT.exists():
    print("Data Root Exists!")
else: 
    print("Data Root doesn't exist!")

Data Root Exists!


In [4]:
df = pd.read_csv(ML_DATA_ROOT)

df

,time,temp,humi,pres,hour,dayofweek,hour_sin,hour_cos,temp_lag1,temp_lag2,temp_lag3,humi_lag1,humi_lag2,humi_lag3,pres_lag1,pres_lag2,pres_lag3,temp_next1h,humi_next1h,pres_next1h
0,2026-01-24 20:38:02,28.5,74,1016.934326,20,5,-0.866025,0.500000,28.6,28.5,28.6,74.0,74.0,74.0,1016.859131,1016.802490,1016.746582,28.5,75.0,1017.522461
1,2026-01-24 20:43:02,28.6,74,1016.971436,20,5,-0.866025,0.500000,28.5,28.6,28.5,74.0,74.0,74.0,1016.934326,1016.859131,1016.802490,28.5,75.0,1017.663086
2,2026-01-24 20:48:02,28.9,74,1017.078125,20,5,-0.866025,0.500000,28.6,28.5,28.6,74.0,74.0,74.0,1016.971436,1016.934326,1016.859131,28.5,75.0,1017.696777
3,2026-01-24 20:53:02,28.7,74,1017.150879,20,5,-0.866025,0.500000,28.9,28.6,28.5,74.0,74.0,74.0,1017.078125,1016.971436,1016.934326,28.5,75.0,1017.770752
4,2026-01-24 20:58:02,28.5,74,1017.206299,20,5,-0.866025,0.500000,28.7,28.9,28.6,74.0,74.0,74.0,1017.150879,1017.078125,1016.971436,28.5,75.0,1017.850830
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2177,2026-02-02 09:26:15,38.3,48,1017.980957,9,0,0.707107,-0.707107,38.1,38.3,38.2,49.0,49.0,49.0,1018.028076,1017.984131,1017.997070,38.5,47.0,1018.157959
2178,2026-02-02 09:31:15,38.3,48,1018.037842,9,0,0.707107,-0.707107,38.3,38.1,38.3,48.0,49.0,49.0,1017.980957,1018.028076,1017.984131,38.5,47.0,1018.206055
2179,2026-02-02 09:36:15,38.3,48,1018.024414,9,0,0.707107,-0.707107,38.3,38.3,38.1,48.0,48.0,49.0,1018.037842,1017.980957,1018.028076,38.5,47.0,1018.149414
2180,2026-02-02 09:41:15,38.3,48,1018.083984,9,0,0.707107,-0.707107,38.3,38.3,38.3,48.0,48.0,48.0,1018.024414,1018.037842,1017.980957,38.5,47.0,1018.127930


In [5]:
# Targets 
TARGETS = ["temp_next1h", "humi_next1h", "pres_next1h"]

# Drop rows with missing target(s)
df = df.dropna(subset=TARGETS).reset_index(drop=True)

exclude_cols = set(TARGETS)

# If your dataset has time columns you don't want as raw strings, exclude them:
for maybe_time_col in ["time", "timestamp", "datetime", "date"]:
    if maybe_time_col in df.columns:
        exclude_cols.add(maybe_time_col)

# Keep numeric features only
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
feature_cols = [c for c in numeric_cols if c not in exclude_cols]

X = df[feature_cols].copy()
y = df[TARGETS].copy()


In [7]:
# =========================
# 3) Train / Val / Test split (time-based)
# =========================
n = len(df)
train_end = int(n * 0.70)
val_end   = int(n * 0.85)  # train 70%, val 15%, test 15%

X_train, y_train = X.iloc[:train_end], y.iloc[:train_end]
X_val,   y_val   = X.iloc[train_end:val_end], y.iloc[train_end:val_end]
X_test,  y_test  = X.iloc[val_end:], y.iloc[val_end:]

print(f"Total rows: {n}")
print(f"Train: {len(X_train)}  Val: {len(X_val)}  Test: {len(X_test)}")

# TimeSeries CV inside TRAIN only
tscv = TimeSeriesSplit(n_splits=5)

def report_split(y_true, y_pred, split_name="SPLIT", model_name="Model"):
    mae = mean_absolute_error(y_true, y_pred, multioutput="raw_values")
    rmse = np.sqrt(mean_squared_error(y_true, y_pred, multioutput="raw_values"))
    r2 = r2_score(y_true, y_pred, multioutput="raw_values")

    print(f"\n===== {model_name} on {split_name} =====")
    for i, t in enumerate(TARGETS):
        print(f"{t}:  MAE={mae[i]:.4f}  RMSE={rmse[i]:.4f}  R2={r2[i]:.4f}")
    print(f"Avg MAE: {float(np.mean(mae)):.4f}")

def avg_mae(y_true, y_pred) -> float:
    mae = mean_absolute_error(y_true, y_pred, multioutput="raw_values")
    return float(np.mean(mae))

Total rows: 2182
Train: 1527  Val: 327  Test: 328


### Ridge Model

In [8]:
# =========================
# 4) Ridge: Pipeline + GridSearch (TRAIN only)
# =========================
ridge_pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("model", MultiOutputRegressor(
        Ridge(random_state=42, max_iter=20000)
    ))
])

ridge_param_grid = {
    "model__estimator__alpha": np.logspace(-4, 4, 25)  # robust sweep
}

ridge_search = GridSearchCV(
    estimator=ridge_pipe,
    param_grid=ridge_param_grid,
    scoring="neg_mean_absolute_error",
    cv=tscv,
    n_jobs=-1
)

ridge_search.fit(X_train, y_train)
best_ridge = ridge_search.best_estimator_
best_ridge_alpha = ridge_search.best_params_["model__estimator__alpha"]
print("\nBest Ridge alpha:", best_ridge_alpha)

# Evaluate Ridge on VAL
ridge_val_pred = best_ridge.predict(X_val)
report_split(y_val.values, ridge_val_pred, split_name="VAL", model_name="Ridge")


Best Ridge alpha: 215.44346900318823

===== Ridge on VAL =====
temp_next1h:  MAE=0.5847  RMSE=0.6671  R2=0.3522
humi_next1h:  MAE=0.8879  RMSE=1.0851  R2=0.8246
pres_next1h:  MAE=0.5267  RMSE=0.6301  R2=0.7695
Avg MAE: 0.6664


### ElasticNet

In [9]:
# =========================
# 5) ElasticNet: Pipeline + GridSearch (TRAIN only)
# =========================
enet_pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("model", MultiOutputRegressor(
        ElasticNet(
            random_state=42,
            max_iter=50000,
            tol=1e-4,
            selection="cyclic"
        )
    ))
])

enet_param_grid = {
    "model__estimator__alpha": np.logspace(-4, 2, 18),     # 1e-4 ... 1e2
    "model__estimator__l1_ratio": [0.05, 0.1, 0.2, 0.35, 0.5, 0.7, 0.9]
}

enet_search = GridSearchCV(
    estimator=enet_pipe,
    param_grid=enet_param_grid,
    scoring="neg_mean_absolute_error",
    cv=tscv,
    n_jobs=-1
)

enet_search.fit(X_train, y_train)
best_enet = enet_search.best_estimator_
print("\nBest ElasticNet params:", enet_search.best_params_)

# Evaluate ElasticNet on VAL
enet_val_pred = best_enet.predict(X_val)
report_split(y_val.values, enet_val_pred, split_name="VAL", model_name="ElasticNet")


Best ElasticNet params: {'model__estimator__alpha': np.float64(0.06660846290809161), 'model__estimator__l1_ratio': 0.9}

===== ElasticNet on VAL =====
temp_next1h:  MAE=0.4811  RMSE=0.5455  R2=0.5668
humi_next1h:  MAE=0.9008  RMSE=1.1164  R2=0.8144
pres_next1h:  MAE=0.5307  RMSE=0.6365  R2=0.7648
Avg MAE: 0.6375


In [10]:
# =========================
# 6) Select model using VAL performance
# =========================
ridge_val_mae = avg_mae(y_val.values, ridge_val_pred)
enet_val_mae  = avg_mae(y_val.values, enet_val_pred)

if ridge_val_mae <= enet_val_mae:
    selected_name = "ridge"
    selected_model = best_ridge
else:
    selected_name = "elasticnet"
    selected_model = best_enet

print(f"\nSelected model (by VAL avg MAE): {selected_name.upper()}")
print(f"Ridge VAL avg MAE: {ridge_val_mae:.4f}")
print(f"Enet  VAL avg MAE: {enet_val_mae:.4f}")


Selected model (by VAL avg MAE): ELASTICNET
Ridge VAL avg MAE: 0.6664
Enet  VAL avg MAE: 0.6375


In [11]:
# =========================
# 7) Refit selected model on TRAIN+VAL (best practice), then test
# =========================
X_trainval = pd.concat([X_train, X_val], axis=0)
y_trainval = pd.concat([y_train, y_val], axis=0)

selected_model.fit(X_trainval, y_trainval)

# Evaluate on TEST
test_pred = selected_model.predict(X_test)
report_split(y_test.values, test_pred, split_name="TEST", model_name=selected_name.upper())


===== ELASTICNET on TEST =====
temp_next1h:  MAE=0.2849  RMSE=0.3679  R2=0.7898
humi_next1h:  MAE=0.7788  RMSE=0.9512  R2=0.7631
pres_next1h:  MAE=0.4991  RMSE=0.5864  R2=0.7630
Avg MAE: 0.5209


In [12]:
# ----- Baseline: persistence (next hour = current value) -----
# Assuming your features include current readings named temp, humi, pres
# If not, use the closest available like temp_lag1 as "current".
def persistence_baseline(df_features, y_true, use="current"):
    # prefer current if present; else fall back to lag1
    temp_col = "temp" if "temp" in df_features.columns else "temp_lag1"
    humi_col = "humi" if "humi" in df_features.columns else "humi_lag1"
    pres_col = "pres" if "pres" in df_features.columns else "pres_lag1"

    y_pred = np.column_stack([
        df_features[temp_col].values,
        df_features[humi_col].values,
        df_features[pres_col].values
    ])
    return y_pred

baseline_test_pred = persistence_baseline(X_test, y_test.values)
report_split(y_test.values, baseline_test_pred, split_name="TEST", model_name="PERSISTENCE BASELINE")


===== PERSISTENCE BASELINE on TEST =====
temp_next1h:  MAE=0.2555  RMSE=0.3406  R2=0.8198
humi_next1h:  MAE=0.6768  RMSE=0.9969  R2=0.7398
pres_next1h:  MAE=0.5426  RMSE=0.6325  R2=0.7243
Avg MAE: 0.4916


### Tree-Based Models

In [25]:
%pip install xgboost

  Using cached xgboost-3.2.0-py3-none-macosx_12_0_arm64.whl.metadata (2.1 kB)
Using cached xgboost-3.2.0-py3-none-macosx_12_0_arm64.whl (2.3 MB)

[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [28]:
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.ensemble import HistGradientBoostingRegressor
from xgboost import XGBRegressor

In [14]:
# =========================
# 1) Load dataset
# =========================
CSV_PATH = "/Users/saikeerthan/Coding/NYP/IOTA/IoT_Weather_project/model-training/datasets/weather_ml_ready.csv"
df = pd.read_csv(CSV_PATH)

TARGETS = ["temp_next1h", "humi_next1h", "pres_next1h"]
df = df.dropna(subset=TARGETS).reset_index(drop=True)

# =========================
# 2) Features = numeric cols except targets (and exclude raw time cols if present)
# =========================
exclude_cols = set(TARGETS)
for maybe_time_col in ["time", "timestamp", "datetime", "date"]:
    if maybe_time_col in df.columns:
        exclude_cols.add(maybe_time_col)

numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
feature_cols = [c for c in numeric_cols if c not in exclude_cols]

X = df[feature_cols].copy()
y = df[TARGETS].copy()


In [15]:
# =========================
# 3) Train / Val / Test split (time-based)
# =========================
n = len(df)
train_end = int(n * 0.70)
val_end   = int(n * 0.85)

X_train, y_train = X.iloc[:train_end], y.iloc[:train_end]
X_val,   y_val   = X.iloc[train_end:val_end], y.iloc[train_end:val_end]
X_test,  y_test  = X.iloc[val_end:], y.iloc[val_end:]

print(f"Rows: total={n} train={len(X_train)} val={len(X_val)} test={len(X_test)}")

tscv = TimeSeriesSplit(n_splits=5)


def report_split(y_true, y_pred, split_name="SPLIT", model_name="Model"):
    mae = mean_absolute_error(y_true, y_pred, multioutput="raw_values")
    rmse = np.sqrt(mean_squared_error(y_true, y_pred, multioutput="raw_values"))
    r2 = r2_score(y_true, y_pred, multioutput="raw_values")

    print(f"\n===== {model_name} on {split_name} =====")
    for i, t in enumerate(TARGETS):
        print(f"{t}:  MAE={mae[i]:.4f}  RMSE={rmse[i]:.4f}  R2={r2[i]:.4f}")
    print(f"Avg MAE: {float(np.mean(mae)):.4f}")


def avg_mae(y_true, y_pred) -> float:
    return float(np.mean(mean_absolute_error(y_true, y_pred, multioutput="raw_values")))

Rows: total=2182 train=1527 val=327 test=328


In [16]:
# =========================
# 4) Persistence baseline (strong baseline)
# =========================
def persistence_baseline(df_features: pd.DataFrame):
    # prefer current columns if present; else lag1 as proxy
    temp_col = "temp" if "temp" in df_features.columns else "temp_lag1"
    humi_col = "humi" if "humi" in df_features.columns else "humi_lag1"
    pres_col = "pres" if "pres" in df_features.columns else "pres_lag1"

    return np.column_stack([
        df_features[temp_col].values,
        df_features[humi_col].values,
        df_features[pres_col].values
    ])

base_test_pred = persistence_baseline(X_test)
report_split(y_test.values, base_test_pred, split_name="TEST", model_name="PERSISTENCE BASELINE")
baseline_test_mae = avg_mae(y_test.values, base_test_pred)


===== PERSISTENCE BASELINE on TEST =====
temp_next1h:  MAE=0.2555  RMSE=0.3406  R2=0.8198
humi_next1h:  MAE=0.6768  RMSE=0.9969  R2=0.7398
pres_next1h:  MAE=0.5426  RMSE=0.6325  R2=0.7243
Avg MAE: 0.4916


In [17]:
# =========================
# 5) Models + robust grids (not crazy big)
# =========================
candidates = []

# A) HistGradientBoosting (fast + strong)
hgb = MultiOutputRegressor(
    HistGradientBoostingRegressor(random_state=42)
)
hgb_grid = {
    "estimator__learning_rate": [0.03, 0.06, 0.1],
    "estimator__max_depth": [3, 5, None],
    "estimator__max_leaf_nodes": [31, 63, 127],
    "estimator__min_samples_leaf": [10, 20, 40],
    "estimator__l2_regularization": [0.0, 0.1, 1.0],
}
candidates.append(("HGB", hgb, hgb_grid))

# B) ExtraTrees (often excellent on tabular)
etr = MultiOutputRegressor(
    ExtraTreesRegressor(random_state=42, n_jobs=-1)
)
etr_grid = {
    "estimator__n_estimators": [300, 600],
    "estimator__max_depth": [None, 10, 20],
    "estimator__min_samples_leaf": [1, 2, 5],
    "estimator__max_features": ["sqrt", 0.7, 1.0],
}
candidates.append(("EXTRATREES", etr, etr_grid))

# C) RandomForest (solid baseline)
rf = MultiOutputRegressor(
    RandomForestRegressor(random_state=42, n_jobs=-1)
)
rf_grid = {
    "estimator__n_estimators": [300, 600],
    "estimator__max_depth": [None, 10, 20],
    "estimator__min_samples_leaf": [1, 2, 5],
    "estimator__max_features": ["sqrt", 0.7, 1.0],
}
candidates.append(("RANDOMFOREST", rf, rf_grid))


In [18]:

# =========================
# 6) Tune each model on TRAIN (TimeSeriesSplit), choose by VAL
# =========================
best_name = None
best_model = None
best_val_mae = float("inf")

for name, model, grid in candidates:
    print(f"\n--- Tuning {name} (TRAIN only) ---")

    search = GridSearchCV(
        estimator=model,
        param_grid=grid,
        scoring="neg_mean_absolute_error",
        cv=tscv,
        n_jobs=-1,
        verbose=0
    )
    search.fit(X_train, y_train)

    tuned = search.best_estimator_
    print("Best params:", search.best_params_)

    val_pred = tuned.predict(X_val)
    report_split(y_val.values, val_pred, split_name="VAL", model_name=name)
    val_mae = avg_mae(y_val.values, val_pred)

    if val_mae < best_val_mae:
        best_val_mae = val_mae
        best_name = name
        best_model = tuned

print(f"\nSelected by VAL avg MAE: {best_name} (VAL avg MAE={best_val_mae:.4f})")


--- Tuning HGB (TRAIN only) ---
Best params: {'estimator__l2_regularization': 0.1, 'estimator__learning_rate': 0.03, 'estimator__max_depth': 5, 'estimator__max_leaf_nodes': 31, 'estimator__min_samples_leaf': 40}

===== HGB on VAL =====
temp_next1h:  MAE=0.5112  RMSE=0.6213  R2=0.4382
humi_next1h:  MAE=0.7049  RMSE=0.9611  R2=0.8624
pres_next1h:  MAE=0.3150  RMSE=0.3914  R2=0.9110
Avg MAE: 0.5103

--- Tuning EXTRATREES (TRAIN only) ---


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Best params: {'estimator__max_depth': None, 'estimator__max_features': 0.7, 'estimator__min_samples_leaf': 5, 'estimator__n_estimators': 600}

===== EXTRATREES on VAL =====
temp_next1h:  MAE=0.3585  RMSE=0.4846  R2=0.6582
humi_next1h:  MAE=0.6618  RMSE=0.9080  R2=0.8772
pres_next1h:  MAE=0.2670  RMSE=0.3345  R2=0.9350
Avg MAE: 0.4291

--- Tuning RANDOMFOREST (TRAIN only) ---
Best params: {'estimator__max_depth': 20, 'estimator__max_features': 'sqrt', 'estimator__min_samples_leaf': 2, 'estimator__n_estimators': 300}

===== RANDOMFOREST on VAL =====
temp_next1h:  MAE=0.3597  RMSE=0.4933  R2=0.6457
humi_next1h:  MAE=0.7934  RMSE=1.0704  R2=0.8294
pres_next1h:  MAE=0.2677  RMSE=0.3281  R2=0.9375
Avg MAE: 0.4736

Selected by VAL avg MAE: EXTRATREES (VAL avg MAE=0.4291)


In [19]:

# =========================
# 7) Refit selected on TRAIN+VAL, evaluate on TEST
# =========================
X_trainval = pd.concat([X_train, X_val], axis=0)
y_trainval = pd.concat([y_train, y_val], axis=0)

best_model.fit(X_trainval, y_trainval)

test_pred = best_model.predict(X_test)
report_split(y_test.values, test_pred, split_name="TEST", model_name=best_name)
tree_test_mae = avg_mae(y_test.values, test_pred)

print(f"\nBaseline TEST avg MAE: {baseline_test_mae:.4f}")
print(f"{best_name} TEST avg MAE: {tree_test_mae:.4f}")

if tree_test_mae < baseline_test_mae:
    print("✅ Tree model beats persistence baseline overall on TEST.")
else:
    print("⚠️ Tree model does NOT beat persistence baseline overall on TEST (still may beat it for one target).")



===== EXTRATREES on TEST =====
temp_next1h:  MAE=0.2385  RMSE=0.3105  R2=0.8503
humi_next1h:  MAE=0.7612  RMSE=0.9387  R2=0.7693
pres_next1h:  MAE=0.3088  RMSE=0.3938  R2=0.8931
Avg MAE: 0.4361

Baseline TEST avg MAE: 0.4916
EXTRATREES TEST avg MAE: 0.4361
✅ Tree model beats persistence baseline overall on TEST.


#### XGBoost

In [29]:
# =========================
# 5) XGBoost model + tuning (TRAIN only)
# =========================
# Good defaults for tabular regression; fast + robust.
xgb_base = XGBRegressor(
    objective="reg:squarederror",
    tree_method="hist",          # fast on CPU
    random_state=42,
    n_jobs=-1,
    eval_metric="mae",
    verbosity=0
)

xgb = MultiOutputRegressor(xgb_base)

# A compact but strong grid (won't take forever)
param_grid = {
    "estimator__n_estimators": [300, 600, 900],
    "estimator__learning_rate": [0.03, 0.05, 0.08],
    "estimator__max_depth": [3, 4, 6],
    "estimator__subsample": [0.7, 0.9, 1.0],
    "estimator__colsample_bytree": [0.7, 0.9, 1.0],
    "estimator__min_child_weight": [1, 5, 10],
    "estimator__reg_lambda": [1.0, 5.0, 10.0],   # L2
}

search = GridSearchCV(
    estimator=xgb,
    param_grid=param_grid,
    scoring="neg_mean_absolute_error",
    cv=tscv,
    n_jobs=-1,
    verbose=0
)

print("\n--- Tuning XGB (TRAIN only) ---")
search.fit(X_train, y_train)

best_xgb = search.best_estimator_
print("Best XGB params:", search.best_params_)

# VAL
val_pred = best_xgb.predict(X_val)
report_split(y_val.values, val_pred, split_name="VAL", model_name="XGB")


--- Tuning XGB (TRAIN only) ---
Best XGB params: {'estimator__colsample_bytree': 0.7, 'estimator__learning_rate': 0.08, 'estimator__max_depth': 3, 'estimator__min_child_weight': 1, 'estimator__n_estimators': 300, 'estimator__reg_lambda': 10.0, 'estimator__subsample': 0.7}

===== XGB on VAL =====
temp_next1h:  MAE=0.4069  RMSE=0.5845  R2=0.5027
humi_next1h:  MAE=0.7682  RMSE=1.0914  R2=0.8226
pres_next1h:  MAE=0.3107  RMSE=0.3819  R2=0.9153
Avg MAE: 0.4953


In [30]:
# =========================
# 6) Refit on TRAIN+VAL, evaluate on TEST
# =========================
X_trainval = pd.concat([X_train, X_val], axis=0)
y_trainval = pd.concat([y_train, y_val], axis=0)

best_xgb.fit(X_trainval, y_trainval)

test_pred = best_xgb.predict(X_test)
report_split(y_test.values, test_pred, split_name="TEST", model_name="XGB")
xgb_test_mae = avg_mae(y_test.values, test_pred)

print(f"\nBaseline TEST avg MAE: {baseline_test_mae:.4f}")
print(f"XGB TEST avg MAE: {xgb_test_mae:.4f}")

if xgb_test_mae < baseline_test_mae:
    print("✅ XGB beats persistence baseline overall on TEST.")
else:
    print("⚠️ XGB does NOT beat persistence baseline overall on TEST.")


===== XGB on TEST =====
temp_next1h:  MAE=0.2347  RMSE=0.3632  R2=0.7952
humi_next1h:  MAE=0.7568  RMSE=0.9473  R2=0.7651
pres_next1h:  MAE=0.2801  RMSE=0.3400  R2=0.9204
Avg MAE: 0.4239

Baseline TEST avg MAE: 0.4916
XGB TEST avg MAE: 0.4239
✅ XGB beats persistence baseline overall on TEST.


In [ ]:
"""
Good direction to discuss first.

A practical way is to make Weather Overview a composite score (0–100) from the latest sensor values plus short-term stability.

Suggested model (rule-based, not ML)

Comfort Score (55%)
Based on temperature + humidity together (use apparent comfort logic).
Best zone gets high score, too hot/cold or too humid/dry lowers it.
Wind Score (25%)
Gentle wind = higher score.
Very low or very high wind lowers score (high wind penalized more).
Stability Score (20%)
Use last 1–3 hours.
If temp/humidity/wind are changing smoothly, higher score.
Sudden swings reduce score.
Then:

Overview = 0.55*Comfort + 0.25*Wind + 0.20*Stability
Clamp to 0–100.
Status label mapping

80–100: Optimal
60–79: Good
40–59: Watch
<40: Alert
Why this is best for your project

Deterministic and explainable (easy for report/demo).
No retraining needed.
Stable UI behavior if you smooth with a rolling window (for example last 3 points at 5-minute intervals).
If you want, next we can agree exact comfort ranges for your climate (tropical vs temperate), then I’ll implement it exactly in the UI/backend.

"""

### DeepLearning Model

In [2]:
import os
import math
import time
import copy
import argparse
from dataclasses import dataclass

import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [6]:

# -----------------------------
# Config
# -----------------------------
@dataclass
class CFG:
    # Data
    csv_path: str = "/Users/saikeerthan/Coding/NYP/IOTA/IoT_Weather_project/model-training/datasets/weather_ml_ready.csv"   # <-- change this to your local file path
    datetime_col: str = "datetime"           # if your csv uses a different name, change here
    lookback_steps: int = 72                 # 72 * 5min = 6 hours
    horizon_steps: int = 12                  # 12 * 5min = 60 minutes
    # features & targets (edit if your column names differ)
    feature_cols: tuple = ("temp", "humi", "pres", "hour_sin", "hour_cos", "dayofweek")
    target_cols: tuple = ("temp_next1h", "humi_next1h", "pres_next1h")

    # Split fractions (time-based)
    train_frac: float = 0.70
    val_frac: float = 0.15  # test = remainder

    # Training
    batch_size: int = 64
    num_workers: int = 0
    epochs: int = 60
    lr: float = 2e-3
    weight_decay: float = 1e-4
    dropout: float = 0.15
    seed: int = 42
    patience: int = 10

    # Model
    tcn_channels: int = 64
    tcn_kernel: int = 3
    tcn_dilations: tuple = (1, 2, 4, 8)
    gru_hidden: int = 64

    # Loss weights for multi-task
    w_temp: float = 0.4
    w_humi: float = 0.3
    w_pres: float = 0.3

    # Output
    out_dir: str = "runs_weather_dl"
    best_model_name: str = "best_model.pt"


def set_seed(seed: int):
    import random
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)


# -----------------------------
# Dataset: sliding windows
# -----------------------------
class WeatherSeqDataset(Dataset):
    def __init__(self, X_seq: np.ndarray, y: np.ndarray):
        """
        X_seq: (N, L, F)
        y:     (N, 3)
        """
        self.X_seq = torch.from_numpy(X_seq).float()
        self.y = torch.from_numpy(y).float()

    def __len__(self):
        return self.X_seq.shape[0]

    def __getitem__(self, idx):
        return self.X_seq[idx], self.y[idx]


def build_windows(df: pd.DataFrame, feature_cols, target_cols, lookback_steps, horizon_steps):
    """
    Creates samples where:
      X = features[t-lookback+1 : t+1]  (length lookback_steps)
      y = targets at time t (already aligned as next1h targets)
    BUT to avoid leakage, we ensure that the targets correspond to t+1h from the *end* of window.
    Since your CSV has *_next1h already, we align y at the window end index.

    Returns:
      X_seq: (N, lookback_steps, F)
      y:     (N, 3)
    """
    feats = df.loc[:, feature_cols].to_numpy()
    tars = df.loc[:, target_cols].to_numpy()

    L = lookback_steps
    # We need the window end index 'end' such that targets are valid there.
    # If your *_next1h was computed as shift(-12), then the last 12 rows may be NaN.
    # We'll drop NaNs earlier, but still guard.
    X_list, y_list = [], []
    for end in range(L - 1, len(df)):
        start = end - (L - 1)
        x = feats[start:end + 1]
        y = tars[end]
        if np.any(np.isnan(x)) or np.any(np.isnan(y)):
            continue
        X_list.append(x)
        y_list.append(y)

    X_seq = np.stack(X_list, axis=0) if len(X_list) else np.empty((0, L, len(feature_cols)))
    y = np.stack(y_list, axis=0) if len(y_list) else np.empty((0, len(target_cols)))
    return X_seq, y

In [7]:
# -----------------------------
# Model: TCN residual blocks + GRU + Attention pooling + 3-head outputs
# -----------------------------
class ResidualTCNBlock(nn.Module):
    def __init__(self, channels, kernel_size, dilation, dropout):
        super().__init__()
        padding = (kernel_size - 1) * dilation  # for causal conv

        self.conv1 = nn.Conv1d(channels, channels, kernel_size, padding=padding, dilation=dilation)
        self.ln1 = nn.LayerNorm(channels)
        self.act1 = nn.GELU()
        self.drop1 = nn.Dropout(dropout)

        self.conv2 = nn.Conv1d(channels, channels, kernel_size, padding=padding, dilation=dilation)
        self.ln2 = nn.LayerNorm(channels)
        self.act2 = nn.GELU()
        self.drop2 = nn.Dropout(dropout)

    def forward(self, x):
        """
        x: (B, C, L)
        Causal trimming after conv because of padding.
        """
        residual = x

        out = self.conv1(x)
        out = out[:, :, :x.size(2)]  # trim to keep causality
        # LayerNorm expects (B, L, C), so transpose
        out = out.transpose(1, 2)
        out = self.ln1(out)
        out = out.transpose(1, 2)

        out = self.act1(out)
        out = self.drop1(out)

        out = self.conv2(out)
        out = out[:, :, :x.size(2)]
        out = out.transpose(1, 2)
        out = self.ln2(out)
        out = out.transpose(1, 2)

        out = self.act2(out)
        out = self.drop2(out)

        return out + residual


class AttentionPooling(nn.Module):
    def __init__(self, hidden_dim):
        super().__init__()
        self.score = nn.Linear(hidden_dim, 1)

    def forward(self, h):
        """
        h: (B, L, H)
        returns: context (B, H)
        """
        # scores: (B, L, 1)
        scores = self.score(h)
        weights = torch.softmax(scores, dim=1)  # over L
        context = torch.sum(weights * h, dim=1)  # (B, H)
        return context


class WeatherForecastModel(nn.Module):
    def __init__(self, num_features, cfg: CFG):
        super().__init__()
        C = cfg.tcn_channels

        self.in_proj = nn.Linear(num_features, C)

        self.tcn_blocks = nn.ModuleList([
            ResidualTCNBlock(C, cfg.tcn_kernel, d, cfg.dropout) for d in cfg.tcn_dilations
        ])

        self.gru = nn.GRU(
            input_size=C,
            hidden_size=cfg.gru_hidden,
            num_layers=1,
            batch_first=True,
            bidirectional=False,
        )
        self.gru_drop = nn.Dropout(cfg.dropout)

        self.attn = AttentionPooling(cfg.gru_hidden)

        # Shared MLP trunk
        self.shared = nn.Sequential(
            nn.Linear(cfg.gru_hidden, 64),
            nn.GELU(),
            nn.Dropout(cfg.dropout),
        )

        # Three heads
        self.temp_head = nn.Sequential(nn.Linear(64, 32), nn.GELU(), nn.Linear(32, 1))
        self.humi_head = nn.Sequential(nn.Linear(64, 32), nn.GELU(), nn.Linear(32, 1))
        self.pres_head = nn.Sequential(nn.Linear(64, 32), nn.GELU(), nn.Linear(32, 1))

    def forward(self, x):
        """
        x: (B, L, F)
        """
        x = self.in_proj(x)  # (B, L, C)

        # TCN wants (B, C, L)
        x = x.transpose(1, 2)
        for block in self.tcn_blocks:
            x = block(x)
        x = x.transpose(1, 2)  # back to (B, L, C)

        h, _ = self.gru(x)  # (B, L, H)
        h = self.gru_drop(h)

        context = self.attn(h)  # (B, H)
        z = self.shared(context)  # (B, 64)

        temp = self.temp_head(z)
        humi = self.humi_head(z)
        pres = self.pres_head(z)

        yhat = torch.cat([temp, humi, pres], dim=1)  # (B, 3)
        return yhat


In [8]:
# -----------------------------
# Metrics
# -----------------------------
@torch.no_grad()
def compute_metrics(y_true, y_pred):
    """
    y_true/y_pred: torch tensors (N,3)
    returns dict of MAE/RMSE per target + mean
    """
    err = y_pred - y_true
    mae = torch.mean(torch.abs(err), dim=0)  # (3,)
    rmse = torch.sqrt(torch.mean(err**2, dim=0) + 1e-12)

    out = {
        "mae_temp": mae[0].item(), "mae_humi": mae[1].item(), "mae_pres": mae[2].item(),
        "rmse_temp": rmse[0].item(), "rmse_humi": rmse[1].item(), "rmse_pres": rmse[2].item(),
        "mae_mean": torch.mean(mae).item(),
        "rmse_mean": torch.mean(rmse).item(),
    }
    return out


@torch.no_grad()
def evaluate(model, loader, device):
    model.eval()
    ys, preds = [], []
    for X, y in loader:
        X = X.to(device)
        y = y.to(device)
        yhat = model(X)
        ys.append(y)
        preds.append(yhat)
    y_true = torch.cat(ys, dim=0)
    y_pred = torch.cat(preds, dim=0)
    return compute_metrics(y_true, y_pred)


In [10]:
# -----------------------------
# Train
# -----------------------------
def train_one_epoch(model, loader, optimizer, loss_fn, device, cfg: CFG):
    model.train()
    running = 0.0
    n = 0
    for X, y in loader:
        X = X.to(device)
        y = y.to(device)

        optimizer.zero_grad(set_to_none=True)
        yhat = model(X)

        # Huber per target, weighted
        loss_temp = loss_fn(yhat[:, 0], y[:, 0])
        loss_humi = loss_fn(yhat[:, 1], y[:, 1])
        loss_pres = loss_fn(yhat[:, 2], y[:, 2])
        loss = cfg.w_temp * loss_temp + cfg.w_humi * loss_humi + cfg.w_pres * loss_pres

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()

        running += loss.item() * X.size(0)
        n += X.size(0)

    return running / max(n, 1)


def main():
    parser = argparse.ArgumentParser()
    parser.add_argument("--csv", type=str, default=None, help="Path to weather_ml_ready.csv")
    parser.add_argument("--datetime_col", type=str, default=None, help="Datetime column name")
    parser.add_argument("--lookback", type=int, default=None, help="Lookback steps (default 72)")
    parser.add_argument("--epochs", type=int, default=None, help="Epochs (default 60)")
    args, unknown = parser.parse_known_args()

    cfg = CFG()
    if args.csv: cfg.csv_path = args.csv
    if args.datetime_col: cfg.datetime_col = args.datetime_col
    if args.lookback: cfg.lookback_steps = args.lookback
    if args.epochs: cfg.epochs = args.epochs

    set_seed(cfg.seed)
    os.makedirs(cfg.out_dir, exist_ok=True)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Device: {device}")

    # -----------------------------
    # Load + clean
    # -----------------------------
    df = pd.read_csv(cfg.csv_path)

    # Parse datetime if present
    if cfg.datetime_col in df.columns:
        df[cfg.datetime_col] = pd.to_datetime(df[cfg.datetime_col], errors="coerce")
        df = df.sort_values(cfg.datetime_col).reset_index(drop=True)
    else:
        # If no datetime column, assume already ordered
        print(f"Warning: datetime_col='{cfg.datetime_col}' not found. Assuming rows are time-ordered.")

    # Keep required columns only
    needed = list(cfg.feature_cols) + list(cfg.target_cols)
    missing = [c for c in needed if c not in df.columns]
    if missing:
        raise ValueError(
            f"Missing columns in CSV: {missing}\n"
            f"Available columns: {list(df.columns)}\n"
            f"Edit CFG.feature_cols / CFG.target_cols / CFG.datetime_col to match your CSV."
        )

    # Drop NaNs in required columns
    df = df.dropna(subset=needed).reset_index(drop=True)

    # -----------------------------
    # Time-based split indices (on rows)
    # We'll build windows after split to avoid mixing times.
    # -----------------------------
    n_total = len(df)
    n_train = int(cfg.train_frac * n_total)
    n_val = int(cfg.val_frac * n_total)
    n_test = n_total - n_train - n_val

    df_train = df.iloc[:n_train].copy()
    df_val = df.iloc[n_train:n_train + n_val].copy()
    df_test = df.iloc[n_train + n_val:].copy()

    print(f"Rows: total={n_total}, train={len(df_train)}, val={len(df_val)}, test={len(df_test)}")

    # -----------------------------
    # Standardize features using TRAIN only
    # -----------------------------
    scaler = StandardScaler()
    scaler.fit(df_train.loc[:, cfg.feature_cols].to_numpy())

    def apply_scaler(d):
        arr = d.loc[:, cfg.feature_cols].to_numpy()
        d.loc[:, cfg.feature_cols] = scaler.transform(arr)
        return d

    df_train = apply_scaler(df_train)
    df_val = apply_scaler(df_val)
    df_test = apply_scaler(df_test)

    # -----------------------------
    # Build windows within each split
    # Important: windows do not cross split boundaries
    # -----------------------------
    X_train, y_train = build_windows(df_train, cfg.feature_cols, cfg.target_cols, cfg.lookback_steps, cfg.horizon_steps)
    X_val, y_val = build_windows(df_val, cfg.feature_cols, cfg.target_cols, cfg.lookback_steps, cfg.horizon_steps)
    X_test, y_test = build_windows(df_test, cfg.feature_cols, cfg.target_cols, cfg.lookback_steps, cfg.horizon_steps)

    if X_train.shape[0] == 0 or X_val.shape[0] == 0:
        raise RuntimeError(
            "No windows were created. This usually means your splits are too small for the lookback length.\n"
            "Try reducing lookback_steps (e.g., 48) or increasing dataset length."
        )

    print(f"Windows: train={X_train.shape}, val={X_val.shape}, test={X_test.shape}")

    train_ds = WeatherSeqDataset(X_train, y_train)
    val_ds = WeatherSeqDataset(X_val, y_val)
    test_ds = WeatherSeqDataset(X_test, y_test)

    train_loader = DataLoader(train_ds, batch_size=cfg.batch_size, shuffle=True, num_workers=cfg.num_workers, drop_last=False)
    val_loader = DataLoader(val_ds, batch_size=cfg.batch_size, shuffle=False, num_workers=cfg.num_workers)
    test_loader = DataLoader(test_ds, batch_size=cfg.batch_size, shuffle=False, num_workers=cfg.num_workers)

    # -----------------------------
    # Model
    # -----------------------------
    model = WeatherForecastModel(num_features=len(cfg.feature_cols), cfg=cfg).to(device)

    optimizer = torch.optim.AdamW(model.parameters(), lr=cfg.lr, weight_decay=cfg.weight_decay)
    loss_fn = nn.HuberLoss(delta=1.0)

    best = {
        "epoch": -1,
        "val_mae_mean": float("inf"),
        "state": None,
    }
    patience_counter = 0

    print("\nStarting training...\n")

    for epoch in range(1, cfg.epochs + 1):
        t0 = time.time()
        train_loss = train_one_epoch(model, train_loader, optimizer, loss_fn, device, cfg)
        val_metrics = evaluate(model, val_loader, device)

        dt = time.time() - t0
        print(
            f"Epoch {epoch:03d} | "
            f"train_loss={train_loss:.4f} | "
            f"val_mae_mean={val_metrics['mae_mean']:.4f} | "
            f"val_rmse_mean={val_metrics['rmse_mean']:.4f} | "
            f"dt={dt:.1f}s"
        )
        print(
            f"  Val MAE: temp={val_metrics['mae_temp']:.4f}, humi={val_metrics['mae_humi']:.4f}, pres={val_metrics['mae_pres']:.4f} | "
            f"RMSE: temp={val_metrics['rmse_temp']:.4f}, humi={val_metrics['rmse_humi']:.4f}, pres={val_metrics['rmse_pres']:.4f}"
        )

        # Early stopping on mean MAE
        if val_metrics["mae_mean"] < best["val_mae_mean"] - 1e-5:
            best["epoch"] = epoch
            best["val_mae_mean"] = val_metrics["mae_mean"]
            best["state"] = copy.deepcopy(model.state_dict())
            patience_counter = 0

            save_path = os.path.join(cfg.out_dir, cfg.best_model_name)
            torch.save(
                {
                    "model_state_dict": best["state"],
                    "scaler_mean": scaler.mean_,
                    "scaler_scale": scaler.scale_,
                    "feature_cols": cfg.feature_cols,
                    "target_cols": cfg.target_cols,
                    "cfg": cfg.__dict__,
                },
                save_path,
            )
            print(f"  ✅ Saved best model to: {save_path}")
        else:
            patience_counter += 1
            if patience_counter >= cfg.patience:
                print(f"\nEarly stopping triggered. Best epoch = {best['epoch']} (val_mae_mean={best['val_mae_mean']:.4f})")
                break

    # Load best and test
    if best["state"] is not None:
        model.load_state_dict(best["state"])

    test_metrics = evaluate(model, test_loader, device)
    print("\n===== TEST RESULTS =====")
    print(
        f"MAE:  temp={test_metrics['mae_temp']:.4f}, humi={test_metrics['mae_humi']:.4f}, pres={test_metrics['mae_pres']:.4f} | mean={test_metrics['mae_mean']:.4f}"
    )
    print(
        f"RMSE: temp={test_metrics['rmse_temp']:.4f}, humi={test_metrics['rmse_humi']:.4f}, pres={test_metrics['rmse_pres']:.4f} | mean={test_metrics['rmse_mean']:.4f}"
    )

    print("\nDone ✅")


if __name__ == "__main__":
    main()

Device: cpu
Rows: total=2182, train=1527, val=327, test=328
Windows: train=(1456, 72, 6), val=(256, 72, 6), test=(257, 72, 6)


/var/folders/_r/50s4djvj3g5_yz8zn0m59nbm0000gn/T/ipykernel_33922/2711983676.py:100: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[ 1.68305752  1.68305752  1.68305752 ... -0.45684992 -0.3595814
 -0.3595814 ]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  d.loc[:, cfg.feature_cols] = scaler.transform(arr)
/var/folders/_r/50s4djvj3g5_yz8zn0m59nbm0000gn/T/ipykernel_33922/2711983676.py:100: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.99117366 0.99117366 0.99117366 ... 0.99117366 0.99117366 0.99117366]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  d.loc[:, cfg.feature_cols] = scaler.transform(arr)
/var/folders/_r/50s4djvj3g5_yz8zn0m59nbm0000gn/T/ipykernel_33922/2711983676.py:100: FutureWarning: Setting an item of incompatible dtype is deprecated and will rais


Starting training...

Epoch 001 | train_loss=332.7471 | val_mae_mean=359.0544 | val_rmse_mean=359.0852 | dt=3.4s
  Val MAE: temp=30.4184, humi=40.2524, pres=1006.4922 | RMSE: temp=30.4269, humi=40.3354, pres=1006.4932
  ✅ Saved best model to: runs_weather_dl/best_model.pt
Epoch 002 | train_loss=312.1450 | val_mae_mean=322.9300 | val_rmse_mean=323.1014 | dt=3.2s
  Val MAE: temp=0.6081, humi=2.3809, pres=965.8009 | RMSE: temp=0.7388, humi=2.7636, pres=965.8018
  ✅ Saved best model to: runs_weather_dl/best_model.pt
Epoch 003 | train_loss=284.8090 | val_mae_mean=301.6325 | val_rmse_mean=301.7746 | dt=3.3s
  Val MAE: temp=5.1861, humi=8.7042, pres=891.0072 | RMSE: temp=5.2358, humi=9.0797, pres=891.0082
  ✅ Saved best model to: runs_weather_dl/best_model.pt
Epoch 004 | train_loss=251.1168 | val_mae_mean=249.2419 | val_rmse_mean=249.4491 | dt=3.3s
  Val MAE: temp=3.0703, humi=2.6897, pres=741.9658 | RMSE: temp=3.1534, humi=3.2269, pres=741.9670
  ✅ Saved best model to: runs_weather_dl/best_

In [13]:
import pandas as pd 

CSV_ROOT = Path("/Users/saikeerthan/Coding/NYP/IOTA/IoT_Weather_project/model-training/datasets/weather_ml_ready.csv")

df = pd.read_csv(CSV_ROOT)

df

,time,temp,humi,pres,hour,dayofweek,hour_sin,hour_cos,temp_lag1,temp_lag2,temp_lag3,humi_lag1,humi_lag2,humi_lag3,pres_lag1,pres_lag2,pres_lag3,temp_next1h,humi_next1h,pres_next1h
0,2026-01-24 20:38:02,28.5,74,1016.934326,20,5,-0.866025,0.500000,28.6,28.5,28.6,74.0,74.0,74.0,1016.859131,1016.802490,1016.746582,28.5,75.0,1017.522461
1,2026-01-24 20:43:02,28.6,74,1016.971436,20,5,-0.866025,0.500000,28.5,28.6,28.5,74.0,74.0,74.0,1016.934326,1016.859131,1016.802490,28.5,75.0,1017.663086
2,2026-01-24 20:48:02,28.9,74,1017.078125,20,5,-0.866025,0.500000,28.6,28.5,28.6,74.0,74.0,74.0,1016.971436,1016.934326,1016.859131,28.5,75.0,1017.696777
3,2026-01-24 20:53:02,28.7,74,1017.150879,20,5,-0.866025,0.500000,28.9,28.6,28.5,74.0,74.0,74.0,1017.078125,1016.971436,1016.934326,28.5,75.0,1017.770752
4,2026-01-24 20:58:02,28.5,74,1017.206299,20,5,-0.866025,0.500000,28.7,28.9,28.6,74.0,74.0,74.0,1017.150879,1017.078125,1016.971436,28.5,75.0,1017.850830
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2177,2026-02-02 09:26:15,38.3,48,1017.980957,9,0,0.707107,-0.707107,38.1,38.3,38.2,49.0,49.0,49.0,1018.028076,1017.984131,1017.997070,38.5,47.0,1018.157959
2178,2026-02-02 09:31:15,38.3,48,1018.037842,9,0,0.707107,-0.707107,38.3,38.1,38.3,48.0,49.0,49.0,1017.980957,1018.028076,1017.984131,38.5,47.0,1018.206055
2179,2026-02-02 09:36:15,38.3,48,1018.024414,9,0,0.707107,-0.707107,38.3,38.3,38.1,48.0,48.0,49.0,1018.037842,1017.980957,1018.028076,38.5,47.0,1018.149414
2180,2026-02-02 09:41:15,38.3,48,1018.083984,9,0,0.707107,-0.707107,38.3,38.3,38.3,48.0,48.0,48.0,1018.024414,1018.037842,1017.980957,38.5,47.0,1018.127930


In [14]:
df[["pres","pres_next1h"]].describe()

,pres,pres_next1h
count,2182.000000,2182.000000
mean,1016.879020,1016.883789
std,1.145357,1.148706
min,1014.175293,1014.175293
25%,1016.000732,1016.000732
50%,1016.865722,1016.865722
75%,1017.856445,1017.871521
max,1019.418457,1019.418457
